# LRU_cache

## 1. 问题背景

我们在栈的章节学习了如何实现一个简单的计算器，我们考虑这样一种场景：我们需要实现一个计算器的类，这个类要对于输入的字符串计算式(我们假设合法)返回结果。因为计算器的实现不是本lab的主题，所以我们用python中的eval函数代替。

### 定义我们的计算器

In [1]:
#定义我们的“计算器”
class calculator:
    def __init__(self):
        pass
    def calc(self,s):
        assert type(s) == str,'input must be string! Got '+str(type(s))
        try:
            return eval(s)
        except:
            print('input error! ',s)

### 测试一下


In [2]:
calc = calculator()
print(calc.calc('1+3-2'))
print(calc.calc('10*3'))
print(calc.calc('123+(3*4)/2'))

2
30
129.0


我们的计算器能够正常工作，我们稍微复杂化一下场景，我们把这个计算器做成一个服务，不停地有请求来得到字符串的值。此外，这些请求有可能有很多是一模一样的。对于同样的字符串我们重复计算显然是无意义的。

### 考虑一个极端例子，我们同一个字符串计算1000次，需要消耗799ms的时间

In [3]:
%%time
s = '1+'*1000+'1'
for i in range(1000):
    calc.calc(s)

Wall time: 799 ms


### 显然，我们可以将已经有的结果缓存下来，每次字符串来了我们先查询缓存，如果有则返回。
### 特别的，字符串比较也是耗时的工作，我们可以将字符串先进行加密，变成hash值。

In [4]:
import hashlib
#利用hashlib来得到字符串的md5值
def get_hashcode(s):
    md5 = hashlib.md5()
    md5.update(s.encode())
    return md5.hexdigest()
#测试一下
get_hashcode('1+1')

'd96e018f51ea61e5ff2f9c349c5da67d'

### 然后我们为我们的计算器添加缓存

In [5]:
#带缓存的计算器
class cache_calculator:
    def __init__(self):
        #定义缓存
        self.cache = {}
        pass
    def calc(self,s):
        assert type(s) == str,'input must be string! Got '+str(type(s))
        #计算hash值
        hashcode =  get_hashcode(s)
        if hashcode in self.cache: return self.cache[hashcode]
        
        try:
            res = eval(s)
            #更新hash值
            self.cache[hashcode] = res
            return res
        except:
            print('input error! ',s)

### 我们再来测试一下刚才的例子,可以看到，现在只要8ms了！

In [6]:
%%time
calc = cache_calculator()
s = '1+'*1000+'1'
for i in range(1000):
    calc.calc(s)

Wall time: 7.98 ms


再来继续思考问题，如果输入的算式很多呢？特别是在现在的大数据时代，动辄就是上亿的访问。而我们的缓存不可能是无限大小的，因此，一旦我们的缓存满了，就需要有取舍。那么缓存满了怎么办？这就是我们LRU算法需要解决的问题。

LRU也是面试中的超高频考题，其本身问题并不复杂，不过要求对各类数据结构都有良好的掌握和理解，也非常贴合实际问题。

## 2. LRU Cache思想

LRU全称是Least Recently Used，即最近最少使用。其核心思想是假如有一个数据很久都没有被访问过了，说明他未来继续被访问的概率也会很低。那么如果我们的缓存满了，这些低概率访问的缓存内容应该优先被清除。

而相反，如果某个数据一直频繁被访问，那么这个数据缓存能节省的时间也越大，应当被保留。

我们考虑从计算机角度来理解这个事情，我们为每个数据赋予一个优先级，那么如果某个数据一直被访问，他的优先级应当最高，否则如果一直未被访问，他的优先级应该最低，当缓存满了的时候，我们将优先级最低的数据清除出去，将空出来的空间存当前数据。

我们学过优先队列，我们假设我们有这么一个优先队列，所有的数都在优先队列里面，每个数都有一个优先级。注意，这里说的"数"实际上是`<key,value>`的键值对，随着key和value的不同，LRU也能适应各类场景。

我们现在来思考LRU cache的性质：
1. 我们说过缓存是有限制的，我们假设容量上限为`capacity`。
2. 通过我们的`key`，我们应该能够通过这个cache得到对应的`value`。即我们有方法`get(key)`，返回`value`,如果`key`不存在，我们返回`-1`。
3. 如果我们访问了某个`get(key)`，我们应当将这个key的优先级提到最高，因为他刚刚访问过。
4. 对应的，我们也需要有`put(key,value)`方法，用于将对应的键值对放进cache中。
5. 对于`put`方法，我们put进去的`key`，也需要有目前最高的优先级，因为他也是“刚刚”访问过。
6. 如果put时发现cache已经满了，我们需要删除当前优先级最低的键值对。

### 堆思想

我们在前面课程中提到过，我们可以用堆来实现优先队列结构。我们先来考虑如何用堆来实现LRU Cache。

1. 首先我们有一个堆，堆中存储`<key,priority>`二元组，按`priority`大小来排序。
2. 我们另外还有一个`dict`,`dict`中也存储`<key,<value,priority>>`二元组，用于指示`key`对应的二元组是什么。
3. 为了方便计算优先级，我们记录全局当前最大的优先级`max_priority`。开始时为0。
4. 对于`get(key)`方法：
    1. 我们从`dict`中查找`key`对应的二元组。dict查询`O(1)`。
    2. 如果没有找到，返回`-1`
    3. 我们从堆中删除这个二元组。堆删除`O(log(n))`。
    4. 我们修改二元组的`priority`为当前最大值加一，即`max_priority+1`，并将`max_priority`自身加一。`O(1)`。
    5. 将新的二元组插入到堆中。堆插入`O(log(n))`。
5. 对于`put(key,value)`方法:
    1. 先调用`get(key)`，这样如果`key`对应的二元组已存在，就把他放在最靠前的位置。`O(log(n))`。
    2. 如果`dict`中已经存在`key`了，直接修改对应的`value`值并返回。`O(1)`。
    3. 如果`dict`中不存在`key`，建立二元组`<key,max_priority+1>`，并将`max_priority`自身加一。`O(1)`。
    4. 将二元组插入到堆中。`O(log(n))`。
    5. 建立二元组`<key,<value,max_priority>>`，将二元组插入到dict中。`O(1)`。
    
所以我们`put`和`get`方法时间复杂度均为`O(log(n))`

### 链表思想

显然，我们的数据是根据优先级按顺序出cache的，这很像一个线性的过程，只是在这个线性过程中存在着某些要“插队”数据。所以我们在堆思想中用了堆这种非线性结构来解决这个问题。

我们思考一下是否能用线性结构来解决这个问题。假如说我们按照优先级把所有的数据排成一列，最前面的数据优先级最高，最后面的数据优先级最低。
先来思考一下我们的操作：
1. get：我们需要把某一个数据拿出来放在开头，其余数据保持顺序。
2. put：我们需要把一个新数据放在开头，如果长度超限我们需要移除末尾元素。

那么显然对于get，我们把某一个数据从列中拿出来并且放在开头的操作，**链表**是非常合适的。从链表中插入和删除节点复杂度均为O(1)。

那么我们考虑用链表来维护这个cache数据结构，来细化一下所需要的操作：
1. 对于`get(key)`:
    1. 我们需要知道`key`在链表中是否存在以及具体位置。链表本身不支持随机查询，不过我们同样可以用dict来存储`key`对应的节点。
    2. 我们需要从链表中删除一个中间节点，并且保持链表顺序，所以这个中间节点需要同时知道其前驱和后继，这也说明了这个链表需要是一个**双向链表**。
    3. 我们还需要将节点插入到链表头部，所以我们需要知道头部的位置，也即记录头指针`head`。
2. 对于`put(key,value)`:
    1. 我们可以用类似之前的方法来判断链表中是否存在目标节点，如果存在则将其提前。
    2. 如果cache满了，我们需要删除最后一个节点，所以我们需要记录尾指针`tail`。
    3. 同样，如果cache没满。我们也需要在尾部插入节点。
    

接下来我们考虑动手实现一个LRU Cache


## 3. LRU Cache的实现

### 定义链表节点

In [7]:
#首先我们定义链表节点，我们的链表是一个双链表，所以有pre和next两个指针，同时，我们还需要存节点的key和value值
class node:
    def __init__(self,value=0,key=0):
        #前一个指针
        self.pre = None
        #后一个指针
        self.next = None
        #value值
        self.value = value
        #key值
        self.key = key

### 定义LRU类

In [8]:
#我们接下来定义我们的LRU类，我们先看看需要哪些方法

class LRUCache(object):
    #初始化方法，capacity是Cache的容量
    def __init__(self, capacity):
        pass

    #我们需要能够在开头插入节点
    def insert_at_head(self,node):
        pass
    
    #我们同样也需要能够删除最后一个节点
    def erase_at_tail(self):
        pass
    
    #核心的get方法
    def get(self, key):
        pass

    #核心的put方法
    def put(self, key, value):
        pass
            


### 初始化方法

In [9]:
    #先来看我们的初始化部分，我们前面提到过，我们需要一个dict来存储key和node的对应关系，需要head和tail指针
    def __init__(self, capacity):
        #存储key和node对应关系的dict
        self.mp = {}
        #头尾指针
        self.tail = None
        self.head = None
        #我们同样定义最大的缓存容量，以及已用的存储容量
        self.capacity = capacity
        self.count = 0

### 在开头插入一个节点

In [10]:
    def insert_at_head(self,node):
        if self.count == 0:
            #先考虑边界条件，如果count=0，当前节点就是唯一节点，当然是头结点和尾节点了
            self.head = node
            self.tail = node
        else:
            #否则我们先找到头结点，把当前节点链在头结点前面
            node.next = self.head
            self.head.pre = node
            #注意我们需要更新现在的头结点
            self.head = node
        #插入节点了count当然加一，并且把key和node放在我们的dict中
        self.count += 1
        self.mp[node.key] = node

### 在尾部删除一个节点

In [11]:
    def erase_at_tail(self):
        #如果当前没有节点当然也不存在删除最后一个节点的事情了
        if self.count == 0: return 
        
        #取出当前尾部节点
        n_node = self.tail
        if self.count == 1:
            #如果当前只有一个节点，那么删除完就全空了
            self.head = None
            self.tail = None
        else:
            #否则我们把tail节点指向当前节点的前一个节点，并断开连接
            self.tail = n_node.pre
            self.tail.next = None
            
        #最后我们从map中删除当前节点
        n_node.pre = None
        n_node.next = None
        n_key = n_node.key
        self.mp.pop(n_key)
        
        #当然count也要减一
        self.count -= 1

### get方法

In [12]:
    def get(self, key):
        #首先考虑边界，如果key不在dict中，我们直接返回-1
        if not key in self.mp: return -1
        
        #取出key所对应的节点
        n_node = self.mp[key]
        
        #第一种情况，如果这个节点就是头结点了，也不存在将其挪到开头的事情了，我们直接返回value
        if n_node == self.head: return n_node.value
        
        #对于其他情况，我们需要将其从链表中删除，并挪到开头，我们先考虑删除。
        
        #我们先取出其前一个和后一个节点
        pre_node = n_node.pre
        next_node = n_node.next
        
        if n_node == self.tail:
            #如果这个节点是尾部节点，直接调用方法删除尾部节点。
            self.erase_at_tail()
        else:
            #否则我们删除这个节点
            
            #1.先将其前后节点连起来
            pre_node.next = next_node
            next_node.pre = pre_node
            
            #2.删除这个节点
            n_node.pre = None
            n_node.next = None
            n_key = n_node.key
            self.mp.pop(n_key)
            self.count -= 1
        
        #最后，我们将这个节点插入到开头
        self.insert_at_head(n_node)
        
        return n_node.value
    

### put方法

In [13]:
    def put(self, key, value):
        #首先我们调用get方法，如果未返回-1，说明其已经在map中有了。
        #由于我们调用get以后会把这个节点挪到开头，所以这种情况下我们直接修改头结点的值就可以了。
        if self.get(key) != -1:
            self.head.value = value
            return
        #如果容量满了，我们先删除最后一个节点
        if self.count == self.capacity:
            self.erase_at_tail()
        
        #最后我们定义一个新节点，把新节点插入到开头
        n_node = node(value,key)
        self.insert_at_head(n_node)

### 代码全貌，你可以尝试不依赖注释看懂每一个部分的内容。

In [14]:
class LRUCache(object):
    def __init__(self, capacity):
        self.mp = {}
        self.tail = None
        self.head = None
        self.capacity = capacity
        self.count = 0

    def insert_at_head(self,node):
        if self.count == 0:
            self.head = node
            self.tail = node
        else:
            node.next = self.head
            self.head.pre = node
            self.head = node
        self.count += 1
        self.mp[node.key] = node
    
    def erase_at_tail(self):
        if self.count == 0: return 
        n_node = self.tail
        if self.count == 1:
            self.head = None
            self.tail = None
        else:
            self.tail = n_node.pre
            self.tail.next = None
            
        n_node.pre = None
        n_node.next = None
        n_key = n_node.key
        self.mp.pop(n_key)
        
        self.count -= 1
            
    def get(self, key):
        if not key in self.mp: return -1
        
        n_node = self.mp[key]
        if n_node == self.head: return n_node.value
        
        pre_node = n_node.pre
        next_node = n_node.next
        
        if n_node == self.tail:
            self.erase_at_tail()
        else:
            pre_node.next = next_node
            next_node.pre = pre_node
            n_node.pre = None
            n_node.next = None
            n_key = n_node.key
            self.mp.pop(n_key)
            self.count -= 1
        
        
        self.insert_at_head(n_node)
        
        return n_node.value

    def put(self, key, value):
        if self.get(key) != -1:
            self.head.value = value
            return
        
        if self.count == self.capacity:
            self.erase_at_tail()
        
        
        n_node = node(value,key)
        self.insert_at_head(n_node)
            


## 4. 使用LRU Cache

我们现在考虑把LRU用在我们的计算器中。

In [15]:
#先定义最大缓存
max_cache_size = 10

#LRU缓存的计算器
class LRU_calculator:
    def __init__(self):
        #定义缓存
        self.cache = LRUCache(max_cache_size)
        pass
    def calc(self,s):
        assert type(s) == str,'input must be string! Got '+str(type(s))
        #计算hash值
        hashcode =  get_hashcode(s)
        
        #尝试从LRU Cache中取值
        res = self.cache.get(hashcode)
        if res != -1: return res
        
        try:
            res = eval(s)
            #更新LRU
            self.cache.put(hashcode,res)
            return res
        except:
            print('input error! ',s)

可以看到，修改非常的简单。且同样只需要9ms，当然这只是一个重复的例子，并没有完全发挥LRU的作用，你可以尝试用更多的例子来进行测试。

In [16]:
%%time
calc = LRU_calculator()
s = '1+'*1000+'1'
for i in range(1000):
    calc.calc(s)

Wall time: 8.98 ms


## 5.总结

我们介绍了LRU Cache的实现思路，LRU Cache很好地缓解了了当缓存容量有限时的效率问题，其实现也非常明了和简洁。实际上在工业界，LRU也是非常常用的策略，这也是其作为高频面试题的原因之一。掌握LRU也能很好地帮助我们理解计算机的缓存机制，优化我们的代码效率。

和其他部分一样,我们同样留一些小思考：
1. LRU Cache各个操作的时间复杂度是多少？
2. 如果我们不同的key数量非常有限，我们还需要使用LRU么？应该如何优化算法？